In [1]:
import json
import os
import numpy as np
import shutil
import cv2
import pattern.en
import random
import re
copy_folder = "C:/Users/pande/Desktop/latest_2023/copy_JSON/line/"
directory = "C:/Users/pande/Desktop/latest_2023/ICDAR2023_CHARTINFO_UB_UNITEC_PMC_TRAIN/annotations_JSON/line/"
image_dir = "C:/Users/pande/Desktop/latest_2023/ICDAR2023_CHARTINFO_UB_UNITEC_PMC_TRAIN/images/"
# lst_of_files = ["PMC1266380___1471-2458-5-112-1.json"]   #vertical grouped
# lst_of_files = ["PMC1183521___pgen.0010002.g001.json"]   #vertical single
# lst_of_files = ['PMC1087839___1471-2156-6-19-3.json']   # vertical no x_ticks
# lst_of_files = ["PMC1298309___1471-2458-5-115-2.json"]   #vertical stacked
lst_of_files = os.listdir(directory)
# lst_of_files = ['PMC1249583___1471-2458-5-97-1_panel_1.json']   # horizontal groups
# lst_of_files = ['PMC1635563___1471-2458-6-267-1.json']  # horizontal stacked

if os.path.exists(copy_folder):
    shutil.rmtree(copy_folder)
if not os.path.exists(copy_folder):
    os.mkdir(copy_folder)

In [2]:
def legend_len(js):
    if len(js['task5']['output']['legend_pairs'])==0:
        c=0
        for i in js['task3']['output']['text_roles']:
            if i == ['legend_label']:
                c+=1
        return c
    else:
        return len(js['task5']['output']['legend_pairs'])

In [3]:
def legends(js):
    if legend_len(js)>0:
        iden = list()
        legends = list()
        for i in js['task5']['output']['legend_pairs']:
            iden.append(i['id'])
        for j in iden:
            for i in js['task2']['output']['text_blocks']:
                if j==i['id']:
                    legends.append(i['text'])
        return legends

In [4]:
def legend_pos(point,img):
    image = cv2.imread(img)
    h,w = image.shape[0],image.shape[1]
    p_h,p_w = point[0],point[1]
    if(p_h>=0 and p_w>=0 and p_h<h/3 and p_w<w/3):
        return "Upper Left"
    elif(p_h>=0 and p_w>=w/3 and p_h<h/3 and p_w<(2*w)/3):
        return "Upper Center"
    elif(p_h>=0 and p_w>=(2*w)/3 and p_h<h/3 and p_w<w):
        return "Upper Right"
    elif(p_h>=h/3 and p_w>=0 and p_h<(2*h)/3 and p_w<w/3):
        return "Center Left"
    elif(p_h>=h/3 and p_w>=w/3 and p_h<(2*h)/3 and p_w<(2*w)/3):
        return "Center"
    elif(p_h>=h/3 and p_w>=(2*w)/3 and p_h<(2*h)/3 and p_w<w):
        return "Center Right"
    elif(p_h>=(2*h)/3 and p_w>=0 and p_h<h and p_w<w/3):
        return "Lower Left"
    elif(p_h>=(2*h)/3 and p_w>=w/3 and p_h<h and p_w<(2*w)/3):
        return "Lower Center"
    elif(p_h>=(2*h)/3 and p_w>=(2*w)/3 and p_h<h and p_w<w):
        return "Lower Right"

In [5]:
def presence_label(js,is_hori):  # return x_label_presence,x_label,y_label_presence,y_label
    plot_origin = [js['task4']['output']['_plot_bb']['x0'],js['task4']['output']['_plot_bb']['y0']+js['task4']['output']['_plot_bb']['height']]
    ids = list()
    for i in js['task3']['output']['text_roles']:
        if 'axis_title' == i['role']:
            ids.append(i['id'])
    ids_dict = dict()
    for i in js['task2']['output']['text_blocks']:
        if i['id'] in ids:
            ids_dict[i['id']] = [i['polygon']['x0'],i['polygon']['y0']]
    x_label_id = None
    y_label_id = None
    y_label,x_label = None,None
    if is_hori=="yes":
        for i in ids_dict:
            if ids_dict[i][0]<plot_origin[0] and ids_dict[i][1]<plot_origin[1]:
                x_label_id = i
            elif ids_dict[i][0]>plot_origin[0] and ids_dict[i][1]>plot_origin[1]:
                y_label_id = i
    else:
        for i in ids_dict:
            if ids_dict[i][0]<plot_origin[0] and ids_dict[i][1]<plot_origin[1]:
                y_label_id = i
            elif ids_dict[i][0]>plot_origin[0] and ids_dict[i][1]>plot_origin[1]:
                x_label_id = i
    for i in js['task2']['output']['text_blocks']:
        if x_label_id!=None and i['id'] == x_label_id:
            x_label = i['text']
        elif y_label_id!=None and i['id'] == y_label_id:
            y_label = i['text']
    if x_label!=None and y_label==None:
        return True,x_label,False,None
    elif x_label==None and y_label!=None:
        return False,None,True,y_label
    elif x_label!=None and y_label!=None:
        return True,x_label,True,y_label
    else:
        return False,None,False,None

In [6]:
def ticks_lst(is_hori,figure,js,orientation):
    if is_hori == "yes":
        x_id_dic = dict()
        for i in js['task4']['output']['axes']['x-axis']:
            x_id_dic[i['id']] = i['tick_pt']
        if orientation == "top":
            x_id_dic = sorted(x_id_dic.items(), key=lambda x:x[1]['y'])  # from top
        else:
            x_id_dic = sorted(x_id_dic.items(), key=lambda x:x[1]['y'],reverse = True)   #from bottom
        x_ids = [i[0] for i in x_id_dic]
        n_ticks = len(x_ids)
        texts = list()
        for i in x_ids:
            for j in js['task2']['output']['text_blocks']:
                if(i == j['id']):
                    texts.append(j['text'])
        return x_ids,n_ticks,texts
    else:
        x_id_dic = dict()
        for i in js['task4']['output']['axes']['x-axis']:
            x_id_dic[i['id']] = i['tick_pt']
        if orientation == "left":
            x_id_dic = sorted(x_id_dic.items(), key=lambda x:x[1]['x'])  # from left
        else:
            x_id_dic = sorted(x_id_dic.items(), key=lambda x:x[1]['x'],reverse = True)  # from right
        x_ids = [i[0] for i in x_id_dic]
        n_ticks = len(x_ids)
        texts = list()
        for i in x_ids:
            for j in js['task2']['output']['text_blocks']:
                if(i == j['id']):
                    texts.append(j['text'])
        return x_ids,n_ticks,texts

In [7]:
def is_stacked(js,title_horizontal,figure):
    if title_horizontal == "yes":
        visuals = list()
        for i in js['task6']['output']['visual elements'][figure]:
            visuals.append(i)
        visuals = sorted(visuals,key = lambda x:x['y0'])
        visuals_dict = list()
        for i in visuals:
            visuals_dict.append(i['y0'])
        set_visuals = set(visuals_dict)
        if len(set_visuals)!=len(visuals_dict):
            stacked = True
        else:
            stacked = False
        stack_per_tick = list()
        dict_of_elements = dict()
        if stacked:
            for i in visuals_dict:
                if i in dict_of_elements.keys():
                    dict_of_elements[i] += 1
                else:
                    dict_of_elements[i] = 1
            stack_per_tick_dict = list()
            c = 0
            for i in dict_of_elements.values():
                stack_per_tick_dict.append(visuals[c:i+c])
                c=i+c
            stack_per_tick = list()
            for i in stack_per_tick_dict:
                count = 0
                for j in i:
                    if j['width']!=1 and j['width']!=0:
                        count+=1
                stack_per_tick.append(count)
        return stacked,stack_per_tick
    else:
        visuals = list()
        for i in js['task6']['output']['visual elements'][figure]:
            visuals.append(i)
        visuals = sorted(visuals,key = lambda x:x['x0'])
        visuals_dict = list()
        for i in visuals:
            visuals_dict.append(i['x0'])
        set_visuals = set(visuals_dict)
        if len(set_visuals)!=len(visuals_dict):
            stacked = True
        else:
            stacked = False
        dict_of_elements = dict()
        stack_per_tick = list()
        if stacked:
            for i in visuals_dict:
                if i in dict_of_elements.keys():
                    dict_of_elements[i] += 1
                else:
                    dict_of_elements[i] = 1
            stack_per_tick_dict = list()
            c = 0
            for i in dict_of_elements.values():
                stack_per_tick_dict.append(visuals[c:i+c])
                c=i+c
            stack_per_tick = list()
            for i in stack_per_tick_dict:
                count = 0
                for j in i:
                    if j['height']!=1 and j['height']!=0:
                        count+=1
                stack_per_tick.append(count)
        return stacked,stack_per_tick

In [8]:
def y_ticks_lst(js,title_horizontal,orientation,figure):
    if title_horizontal == "yes":
        y_id_dic = dict()
        for i in js['task4']['output']['axes']['y-axis']:
            y_id_dic[i['id']] = i['tick_pt']
        if orientation == "left":
            y_id_dic = sorted(y_id_dic.items(), key=lambda x:x[1]['x'])  # from left
        else:
            y_id_dic = sorted(y_id_dic.items(), key=lambda x:x[1]['x'],reverse = True)  # from right
        y_ids = [i[0] for i in y_id_dic]
        n_ticks = len(y_ids)
        texts = list()
        for i in y_ids:
            for j in js['task2']['output']['text_blocks']:
                if(i == j['id']):
                    texts.append(j['text'])
        return y_ids,n_ticks,texts
    else:
        y_id_dic = dict()
        for i in js['task4']['output']['axes']['y-axis']:
            y_id_dic[i['id']] = i['tick_pt']
        if orientation == "top":
            y_id_dic = sorted(y_id_dic.items(), key=lambda x:x[1]['y'])  # from top
        else:
            y_id_dic = sorted(y_id_dic.items(), key=lambda x:x[1]['y'],reverse = True)  # from bottom
        y_ids = [i[0] for i in y_id_dic]
        n_ticks = len(y_ids)
        texts = list()
        for i in y_ids:
            for j in js['task2']['output']['text_blocks']:
                if(i == j['id']):
                    texts.append(j['text'])
        return y_ids,n_ticks,texts

In [9]:
def groups(js,title_horizontal,orientation,figure):
    if title_horizontal == "yes":
        visual_elements_dict = js['task6']['output']['visual elements'][figure]
        if orientation == 'top':
            visual_elements_dict = sorted(visual_elements_dict,key=lambda x:x['y0'])
        else:
            visual_elements_dict = sorted(visual_elements_dict,key=lambda x:x['y0'],reverse = True)
        if orientation == 'top':
            num_per_grp = [0]*len(visual_elements_dict)
            num_per_grp[0] = 1
            c=0
            for i in range(len(visual_elements_dict)-1):
                if visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']+visual_elements_dict[i]['height']) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']+visual_elements_dict[i]['height']-1) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']+visual_elements_dict[i]['height']+1) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']+visual_elements_dict[i]['height']-2) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']+visual_elements_dict[i]['height']+2):
                    num_per_grp[c]+=1
                else:
                    c+=1
                    num_per_grp[c]=1
        else:
            num_per_grp = [0]*len(visual_elements_dict)
            num_per_grp[0] = 1
            c=0
            for i in range(len(visual_elements_dict)-1):
                if visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']-visual_elements_dict[i]['height']) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']-visual_elements_dict[i]['height']-1) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']-visual_elements_dict[i]['height']+1) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']-visual_elements_dict[i]['height']-2) or \
                visual_elements_dict[i+1]['y0'] == (visual_elements_dict[i]['y0']-visual_elements_dict[i]['height']+2):
                    num_per_grp[c]+=1
                else:
                    c+=1
                    num_per_grp[c]=1
        num_fig_per_grp = list()
        for i in num_per_grp:
            if i!=0:
                num_fig_per_grp.append(i)
        flag = 0
        for i in num_fig_per_grp:
            if i!=0 and i!=1:
                flag = 0
                break
            else:
                flag = 1
        is_grp = True if flag == 0 else False
        if is_grp:
            c = 0
            list_per_grp,figure_per_group = list(),list()
            for i in num_fig_per_grp:
                list_per_grp.append(visual_elements_dict[c:i+c])
                c=i+c
            for i in list_per_grp:
                count = 0
                for j in i:
                    if j['width']!=0 and j['width']!=1:
                        count+=1
                figure_per_group.append(count)
            return is_grp,figure_per_group,len(figure_per_group)
        else:
            return is_grp,num_fig_per_grp,len(num_fig_per_grp)
    else:
        visual_elements_dict = js['task6']['output']['visual elements'][figure]
        if orientation == 'left':
            visual_elements_dict = sorted(visual_elements_dict,key=lambda x:x['x0'])
        else:
            visual_elements_dict = sorted(visual_elements_dict,key=lambda x:x['x0'],reverse = True)
        if orientation == 'left':
            num_per_grp = [0]*len(visual_elements_dict)
            num_per_grp[0] = 1
            c=0
            for i in range(len(visual_elements_dict)-1):
                if visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']+visual_elements_dict[i]['width']) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']+visual_elements_dict[i]['width']-1) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']+visual_elements_dict[i]['width']+1) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']+visual_elements_dict[i]['width']-2) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']+visual_elements_dict[i]['width']+2):
                    num_per_grp[c]+=1
                else:
                    c+=1
                    num_per_grp[c]=1
        else:
            num_per_grp = [0]*len(visual_elements_dict)
            num_per_grp[0] = 1
            c=0
            for i in range(len(visual_elements_dict)-1):
                if visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']-visual_elements_dict[i]['width']) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']-visual_elements_dict[i]['width']-1) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']-visual_elements_dict[i]['width']+1) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']-visual_elements_dict[i]['width']-2) or \
                visual_elements_dict[i+1]['x0'] == (visual_elements_dict[i]['x0']-visual_elements_dict[i]['width']+2):
                    num_per_grp[c]+=1
                else:
                    c+=1
                    num_per_grp[c]=1
        num_fig_per_grp = list()
        for i in num_per_grp:
            if i!=0:
                num_fig_per_grp.append(i)
        flag = 0
        for i in num_fig_per_grp:
            if i!=0 and i!=1:
                flag = 0
                break
            else:
                flag = 1
        is_grp = True if flag == 0 else False
        if is_grp:
            c = 0
            list_per_grp,figure_per_group = list(),list()
            for i in num_fig_per_grp:
                list_per_grp.append(visual_elements_dict[c:i+c])
                c=i+c
            for i in list_per_grp:
                count = 0
                for j in i:
                    if j['height']!=0 and j['height']!=1:
                        count+=1
                figure_per_group.append(count)
            return is_grp,figure_per_group,len(figure_per_group)
        else:
            return is_grp,num_fig_per_grp,len(num_fig_per_grp)

In [10]:
def extrapolate(x_cors,y_cors,x):
    for i in range(len(x_cors)):
        if x <= x_cors[i]:
            m = (y_cors[i] - y_cors[i-1])/(x_cors[i] - x_cors[i-1])
            return m * (x - x_cors[i]) +  y_cors[i]
    raise Exception("The provided x coordinate is outside the range of the data")

In [11]:
for file in lst_of_files:
    with open(directory+file,'r') as f:
        js = json.load(f)
        js['task7'] = dict()
        
        if 'task1' in js.keys():
            if js['task1']!=None:
                title_horizontal = "yes" if "horizontal" in js['task1']['output']['chart_type'].split(" ") else "no"

        if 'task5' in js.keys():
            if js['task5'] != None:
                if legend_len(js)>0:
                    js['task7']['legend_length'] = legend_len(js)
                    js['task7']['legend_presence'] = True
                else:
                    js['task7']['legend_presence'] = False

        if 'task5' in js.keys() and 'task3' in js.keys() and 'task2' in js.keys():
            if js['task5']!=None and js['task3']!=None and js['task2']!=None:
                if(len(js['task5']['output']['legend_pairs'])>=1):   #bounding box for legend
                    y_min,x_min,height,width=100000,100000,0,0
                    for i in js['task5']['output']['legend_pairs']:
                        y_min = min(i['bb']['y0'],y_min)  #min y_coordinate
                        x_min = min(i['bb']['x0'],x_min)  #min x_coordinate
                        height = max(i['bb']['height'],height)  #getting the height for padding
                        width = max(i['bb']['width'],width)  #getting the width for padding
                    x_max,y_max =0,0
                    for i in js['task3']['output']['text_roles']:
                        if(i['role'] == "legend_label"):
                            identity = i['id']
                            for j in js['task2']['output']['text_blocks']:
                                if j["id"]==identity:
                                    x_max = max(j['polygon']['x2'],x_max)
                                    y_max = max(j['polygon']['y2'],y_max)
                    bb=[y_min-height,x_min-width,y_max+height,x_max+width]
                    center = ((bb[0]+bb[2])//2,(bb[1]+bb[3])//2)
                    img = image_dir+directory[92:]+os.path.splitext(file)[0]+".jpg"
                    js['task7']['legend_position'] = legend_pos(center,img)

        if 'task5' in js.keys() and 'task3' in js.keys() and 'task2' in js.keys():
            if js['task5']!=None and js['task3']!=None and js['task2']!=None:
                if legend_len(js)>0:
                    js['task7']['legends'] = legends(js)

        if 'task4' in js.keys() and 'task3' in js.keys() and 'task2' in js.keys() and 'task1' in js.keys():
            if js['task4']!=None and js['task3']!=None and js['task2']!=None and js['task1']!=None:
                title_horizontal = "yes" if "horizontal" in js['task1']['output']['chart_type'].split(" ") else "no"
                p_x,x_label,p_y,y_label = presence_label(js,title_horizontal)
                js['task7']['x_label_presence'] = p_x
                js['task7']['y_label_presence'] = p_y
                if p_x:
                    js['task7']['x_label'] = x_label
                if p_y:
                    js['task7']['y_label'] = y_label

        if 'task1' in js.keys():
            if js['task1']!=None:
                list_figures = ['bar','line','box','scatter']
                for i in list_figures:
                    if(i in js['task1']['output']['chart_type'].split(" ")):
                        if i=="bar":
                            figure_type = "bars"
                        elif i=="box":
                            figure_type = "boxplots"
                        elif i=="line":
                            figure_type = "lines"
                        elif i=="scatter":
                            figure_type = "scatter points"
                js['task7']['figure_type'] = figure_type

        if 'task2' in js.keys() and 'task4' in js.keys():
            if js['task2']!=None and js['task4']!=None:
                if title_horizontal == "yes":
                    orientation = "top"
                    x_ids,n_ticks,texts = ticks_lst(title_horizontal,figure_type,js,orientation)
                else:
                    orientation = 'left'
                    x_ids,n_ticks,texts = ticks_lst(title_horizontal,figure_type,js,orientation)
                if n_ticks!=0:
                    if title_horizontal == "yes":
                        orientation = 'top'
                        x_ids,n_ticks,texts = ticks_lst(title_horizontal,figure_type,js,orientation)
                        js['task7']['x_ticks_from_top'] = texts
                        js['task7']['number_of_x_ticks'] = n_ticks
                        js['task7']['x_tick_id_from_top'] = x_ids

                        orientation = 'bottom'
                        x_ids,n_ticks,texts = ticks_lst(title_horizontal,figure_type,js,orientation)
                        js['task7']['x_ticks_from_bottom'] = texts
                        js['task7']['x_tick_id_from_bottom'] = x_ids
                    else:
                        orientation = 'left'
                        x_ids,n_ticks,texts = ticks_lst(title_horizontal,figure_type,js,orientation)
                        js['task7']['x_ticks_from_left'] = texts
                        js['task7']['number_of_x_ticks'] = n_ticks
                        js['task7']['x_tick_id_from_left'] = x_ids

                        orientation = 'right'
                        x_ids,n_ticks,texts = ticks_lst(title_horizontal,figure_type,js,orientation)
                        js['task7']['x_ticks_from_right'] = texts
                        js['task7']['x_tick_id_from_right'] = x_ids
                else:
                    js['task7']['number_of_x_ticks'] = n_ticks
        
        if 'task5' in js.keys() and 'task1' in js.keys() and 'task3' in js.keys() and 'task6' in js.keys():
            if js['task5']!=None and js['task3']!=None and js['task6']!=None and js['task1']!=None:
                list_figures = ['bar','line','box','scatter']
                for i in list_figures:
                    if(i in js['task1']['output']['chart_type'].split(" ")):
                        if i=="bar":
                            figure_type = "bars"
                        elif i=="box":
                            figure_type = "boxplots"
                        elif i=="line":
                            figure_type = "lines"
                        elif i=="scatter":
                            figure_type = "scatter points"
                if figure_type == "bars":
                    title_horizontal = "yes" if "horizontal" in js['task1']['output']['chart_type'].split(" ") else "no"
                    js['task7']['is_stacked'],dict_of_elements = is_stacked(js,title_horizontal,figure_type)
                    if js['task7']['is_stacked']:
                        if title_horizontal == 'yes':
                            js['task7']['stacks_per_tick_from_top'] = dict_of_elements
                        else:
                            js['task7']['stacks_per_tick_from_left'] = dict_of_elements
                        
        if 'task6' in js.keys():
            if js['task6']!=None:
                data_series = 0
                for i in js['task6']['output']['data series']:
                    if i['name']!=None:
                        data_series+=1
                js['task7']['number_of_data_series'] = data_series
                
        if 'task6' in js.keys():
            if js['task6']!=None:
                name_dat = list()
                for i in js['task6']['output']['data series']:
                    name_dat.append(i['name'])
                js['task7']['name_of_data_series'] = name_dat
                
        if 'task2' in js.keys() and 'task4' in js.keys():
            if js['task2']!=None and js['task4']!=None:
                list_figures = ['bar','line','box','scatter']
                for i in list_figures:
                    if(i in js['task1']['output']['chart_type'].split(" ")):
                        if i=="bar":
                            figure_type = "bars"
                        elif i=="box":
                            figure_type = "boxplots"
                        elif i=="line":
                            figure_type = "lines"
                        elif i=="scatter":
                            figure_type = "scatter points"
                if title_horizontal == "yes":
                    orientation = "left"
                    y_ids,n_y_ticks,y_texts = y_ticks_lst(js,title_horizontal,orientation,figure_type)
                else:
                    orientation = 'top'
                    y_ids,n_y_ticks,y_texts = y_ticks_lst(js,title_horizontal,orientation,figure_type)
                if n_y_ticks!=0:
                    if title_horizontal == "yes":
                        orientation = 'left'
                        y_ids,n_y_ticks,y_texts = y_ticks_lst(js,title_horizontal,orientation,figure_type)
                        js['task7']['y_ticks_from_left'] = y_texts
                        js['task7']['number_of_y_ticks'] = n_y_ticks
                        js['task7']['y_tick_id_from_left'] = y_ids

                        orientation = 'right'
                        y_ids,n_y_ticks,y_texts = y_ticks_lst(js,title_horizontal,orientation,figure_type)
                        js['task7']['y_ticks_from_right'] = y_texts
                        js['task7']['y_tick_id_from_right'] = y_ids
                    else:
                        orientation = 'top'
                        y_ids,n_y_ticks,y_texts = y_ticks_lst(js,title_horizontal,orientation,figure_type)
                        js['task7']['y_ticks_from_top'] = y_texts
                        js['task7']['number_of_y_ticks'] = n_y_ticks
                        js['task7']['y_tick_id_from_top'] = y_ids

                        orientation = 'bottom'
                        y_ids,n_y_ticks,y_texts = y_ticks_lst(js,title_horizontal,orientation,figure_type)
                        js['task7']['y_ticks_from_bottom'] = y_texts
                        js['task7']['y_tick_id_from_bottom'] = y_ids
                else:
                    js['task7']['number_of_y_ticks'] = n_ticks
                    
        if 'task5' in js.keys():
            if js['task5']!=None:
                if legend_len(js)>1:
                    x_coor,y_coor = list(),list()
                    for i in js['task5']['output']['legend_pairs']:
                        x_coor.append(i['bb']['x0'])
                        y_coor.append(i['bb']['y0'])
                    js['task7']['legend_labels_orientation'] = "Horizontal" if all(v in [y_coor[0],y_coor[0]+1,y_coor[0]-1] for v in y_coor) else "Vertical"
                    
        if 'task5' in js.keys() and 'task1' in js.keys() and 'task3' in js.keys() and 'task6' in js.keys():
            if js['task5']!=None and js['task3']!=None and js['task6']!=None and js['task1']!=None:
                list_figures = ['bar','line','box','scatter']
                for i in list_figures:
                    if(i in js['task1']['output']['chart_type'].split(" ")):
                        if i=="bar":
                            figure_type = "bars"
                        elif i=="box":
                            figure_type = "boxplots"
                        elif i=="line":
                            figure_type = "lines"
                        elif i=="scatter":
                            figure_type = "scatter points"
                if figure_type == "bars":
                    if title_horizontal == 'yes':
                        orientation = 'top'
                        is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                        js['task7']['is_grouped'] = is_grp
                        if is_grp and js['task7']['is_stacked']==False:
                            js['task7']['total_number_of_grps'] = num_of_grps
                            orientation = 'top'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_top'] = figure_per_group
                            orientation = 'bottom'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_bottom'] = figure_per_group
                        elif js['task7']['is_stacked']==False and is_grp==False:
                            js['task7']['total_number_of_grps'] = 0
                            orientation = 'top'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_top'] = figure_per_group
                            orientation = 'bottom'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_bottom'] = figure_per_group
                    else:
                        orientation = 'left'
                        is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                        js['task7']['is_grouped'] = is_grp
                        if is_grp and js['task7']['is_stacked']==False:
                            js['task7']['total_number_of_grps'] = num_of_grps
                            orientation = 'left'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_left'] = figure_per_group
                            orientation = 'right'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_right'] = figure_per_group
                        elif js['task7']['is_stacked']==False and is_grp==False:
                            js['task7']['total_number_of_grps'] = 0
                            orientation = 'left'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_left'] = figure_per_group
                            orientation = 'right'
                            is_grp,figure_per_group,num_of_grps = groups(js,title_horizontal,orientation,figure_type)
                            js['task7']['number_of_grps_per_tick_from_right'] = figure_per_group
                            
        if 'task6' in js.keys() and 'task2' in js.keys() and 'task4' in js.keys():
            if js['task6'] != None and js['task2']!=None and js['task4'] != None:
                ls_y_ticks = js['task4']['output']['axes']['y-axis']
                ls_y_ticks = sorted(ls_y_ticks,key = lambda x:x['tick_pt']['y'],reverse = True)
                if len(ls_y_ticks)>0:
                    origin_id = ls_y_ticks[0]['id']
                    for i in js['task2']['output']['text_blocks']:
                        if i['id']==origin_id:
                            origin = re.findall("\d+", i['text'])[0]
                            origin = int(origin)
                    # answer
                    if origin>0:
                        js['task7']['are_there_zero_values'] = "no"
                    elif origin==0:
                        figure = js['task7']['figure_type']
                        flag = 0
                        if figure == "bars" and title_horizontal == "no":
                            for i in js['task6']['output']['visual elements'][figure]:
                                if i['height']==0 or i['height']==1:
                                    flag = 1
                                    break
                            js['task7']['are_there_zero_values'] = 'yes' if flag==1 else 'no'
                        elif figure == "bars" and title_horizontal == "yes":
                            for i in js['task6']['output']['visual elements'][figure]:
                                if i['width']==0 or i['width']==1:
                                    flag = 1
                                    break
                            js['task7']['are_there_zero_values'] = 'yes' if flag==1 else 'no'
                        elif figure == "scatter points":
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    if round(j['y'],3) == 0.000:
                                        flag = 1
                                        break
                            js['task7']['are_there_zero_values'] = 'yes' if flag==1 else 'no'
                        elif figure == "lines":
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    if round(j['y'],2) == 0.00:
                                        flag = 1
                                        break
                            js['task7']['are_there_zero_values'] = 'yes' if flag==1 else 'no'
                    else:
                        figure = js['task7']['figure_type']
                        f=1
                        if figure == "bars" and title_horizontal == "no":
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    if 'y' in j.keys():
                                        if j['y']==0:
                                            f=0
                                            break
                                    else:
                                        if j['y2']==0:
                                            f=0
                                            break
                            js['task7']['are_there_zero_values'] = "yes" if f==0 else "no"
                        elif figure == "bars" and title_horizontal == "yes":
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    if 'y' in j.keys():
                                        if j['y']==0:
                                            f=0
                                            break
                                    else:
                                        if j['y2']==0:
                                            f=0
                                            break
                            js['task7']['are_there_zero_values'] = "yes" if f==0 else "no"
                        elif figure == "scatter points":
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    if round(j['y'],3) == 0.000:
                                        flag = 1
                                        break
                            js['task7']['are_there_zero_values'] = 'yes' if flag==1 else 'no'
                        elif figure == "lines":
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    if round(j['y'],2) == 0.00:
                                        flag = 1
                                        break
                            js['task7']['are_there_zero_values'] = 'yes' if flag==1 else 'no'
                            
        if 'task6' in js.keys() and 'task2' in js.keys() and 'task4' in js.keys():
            if js['task6'] != None and js['task2']!=None and js['task4'] != None:
                figure = js['task7']['figure_type']
                if figure == "lines":
                    js['task7']['number_of_figures'] = len(js['task6']['output']['visual elements'][figure])
                elif figure == "scatter points":
                    count = 0
                    for i in js['task6']['output']['visual elements'][figure]:
                        for j in i:
                            count+=1
                    js['task7']['number_of_figures'] = count
                elif figure == 'bars' and title_horizontal == 'no':
                    if 'task7' in js.keys():
                        if 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                            if js['task7']['is_stacked'] and js['task7']['is_grouped']==False:
                                if 'stacks_per_tick_from_left' in js['task7'].keys():
                                    js['task7']['number_of_figures'] = sum(js['task7']['stacks_per_tick_from_left'])
                            elif js['task7']['is_grouped'] and js['task7']['is_stacked']==False:
                                if 'number_of_grps_per_tick_from_left' in js['task7'].keys():
                                    js['task7']['number_of_figures'] = sum(js['task7']['number_of_grps_per_tick_from_left'])
                            else:
                                if 'number_of_grps_per_tick_from_left' in js['task7'].keys():
                                    js['task7']['number_of_figures'] = sum(js['task7']['number_of_grps_per_tick_from_left'])
                elif figure == 'bars' and title_horizontal == 'yes':
                    if 'task7' in js.keys():
                        if 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                            if js['task7']['is_stacked'] and js['task7']['is_grouped']==False:
                                if 'stacks_per_tick_from_top' in js['task7'].keys():
                                    js['task7']['number_of_figures'] = sum(js['task7']['stacks_per_tick_from_top'])
                            elif js['task7']['is_grouped'] and js['task7']['is_stacked']==False:
                                if 'number_of_grps_per_tick_from_top' in js['task7'].keys():
                                    js['task7']['number_of_figures'] = sum(js['task7']['number_of_grps_per_tick_from_top'])
                            else:
                                if 'number_of_grps_per_tick_from_top' in js['task7'].keys():
                                    js['task7']['number_of_figures'] = sum(js['task7']['number_of_grps_per_tick_from_top'])
                                    
        if 'figure_type' in js['task7'].keys() and 'task3' in js.keys() and 'task2' in js.keys():
            figure = js['task7']['figure_type']
            if js['task3']!=None and js['task2']!=None:
                if figure == "lines":
                    count = 0
                    iden = list()
                    for i in js['task3']['output']['text_roles']:
                        if i['role'] == 'mark_label':
                            count+=1
                            iden.append(i['id'])
                    if count>0:
                        js['task7']['number_of_mark_label'] = count
                    marks = list()
                    if len(iden)!=0:
                        for i in iden:
                            for j in js['task2']['output']['text_blocks']:
                                if j['id'] == i:
                                    marks.append(j['text'])
                        js['task7']['mark_labels_lst'] = marks
                        
        if 'task2' in js.keys() and 'task3' in js.keys():
            if js['task2']!=None and js['task3']!=None:
                identity=list()
                ans = ""
                for i in js['task3']['output']['text_roles']:
                    if(i['role']=='chart_title'):
                        identity.append(i['id'])
                if len(identity)>0:
                    for i in identity:
                        for j in js['task2']['output']['text_blocks']:
                            if j['id'] == i:
                                ans = j['text']
                if ans!="":
                    js['task7']['chart_title'] = ans
        
        if 'task6' in js.keys():
            if js['task7']['figure_type'] == 'lines':
                if 'x_ticks_from_left' in js['task7'].keys():
                    if len(js['task7']['x_ticks_from_left'])!=0:
                        ticks = js['task7']['x_ticks_from_left']
                        try:
                            tick_int = list()
                            for i in ticks:
                                tick_int.append(float(i))
                            if 'mark_labels_lst' in js['task7'].keys():
                                marks = js['task7']['mark_labels_lst']
                                marks_pts = dict()
                                marks_extra = dict()
                                for i in marks:
                                    x_cors,y_cors = list(),list()
                                    for j in js['task6']['output']['data series']:
                                        if j['name'] == i:
                                            for k in j['data']:
                                                x_cors.append(k['x'])
                                                y_cors.append(k['y'])
                                    marks_pts[i] = [x_cors,y_cors]
                                for i in marks_pts:
                                    x_cors,y_cors = marks_pts[i][0],marks_pts[i][1]
                                    pts = list()
                                    for j in tick_int:
                                        y = extrapolate(x_cors,y_cors,j)
                                        pts.append([j,y])
                                    marks_extra[i] = pts
                                js['task7']['extrapolated'] = marks_extra
                            if 'legends' in js['task7'].keys():
                                legs = js['task7']['legends']
                                leg_pts = dict()
                                leg_extra = dict()
                                for i in legs:
                                    x_cors,y_cors = list(),list()
                                    for j in js['task6']['output']['data series']:
                                        if j['name'] == i:
                                            for k in j['data']:
                                                x_cors.append(k['x'])
                                                y_cors.append(k['y'])
                                    leg_pts[i] = [x_cors,y_cors]
                                for i in leg_pts:
                                    x_cors,y_cors = leg_pts[i][0],leg_pts[i][1]
                                    pts = list()
                                    for j in tick_int:
                                        y = extrapolate(x_cors,y_cors,j)
                                        pts.append([j,y])
                                    leg_extra[i] = pts
                                js['task7']['extrapolated'] = leg_extra
                            if 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                                x_cors,y_cors = list(),list()
                                for i in js['task6']['output']['data series']:
                                    name = i['name']
                                    for j in i['data']:
                                        x_cors.append(j['x'])
                                        y_cors.append(j['y'])
                                extra = dict()
                                extra_pts = list()
                                for i in tick_int:
                                    y = extrapolate(x_cors,y_cors,i)
                                    extra_pts.append([i,y])
                                extra[name] = extra_pts
                                js['task7']['extrapolated'] = extra
                            if 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])>1:
                                leg_pts = dict()
                                leg_extra = dict()
                                for i in js['task7']['name_of_data_series']:
                                    x_coor,y_coor = list(),list()
                                    for j in js['task6']['output']['data series']:
                                        if j['name'] == i:
                                            for k in j['data']:
                                                x_coor.append(k['x'])
                                                y_coor.append(k['y'])
                                    leg_pts[i] = [x_coor,y_coor]
                                leg_extra = dict()
                                for i in leg_pts:
                                    x_cors,y_cors = leg_pts[i][0],leg_pts[i][1]
                                    pts = list()
                                    for j in tick_int:
                                        y = extrapolate(x_cors,y_cors,j)
                                        pts.append([j,y])
                                    leg_extra[i] = pts
                                js['task7']['extrapolated'] = leg_extra
                        except:
                            pass
                        
        if 'task6' in js.keys():
            if js['task7']['figure_type'] == 'scatter points':
                if 'x_ticks_from_left' in js['task7'].keys():
                    if len(js['task7']['x_ticks_from_left'])!=0:
                        ticks = js['task7']['x_ticks_from_left']
                        try:
                            tick_int = list()
                            for i in ticks:
                                tick_int.append(float(i))
                            if 'legends' in js['task7'].keys():
                                legs = js['task7']['legends']
                                leg_pts = dict()
                                leg_extra = dict()
                                for i in legs:
                                    x_cors,y_cors = list(),list()
                                    for j in js['task6']['output']['data series']:
                                        if j['name'] == i:
                                            for k in j['data']:
                                                x_cors.append(k['x'])
                                                y_cors.append(k['y'])
                                    leg_pts[i] = [x_cors,y_cors]
                                for i in leg_pts:
                                    x_cors,y_cors = leg_pts[i][0],leg_pts[i][1]
                                    pts = list()
                                    for j in tick_int:
                                        y = extrapolate(x_cors,y_cors,j)
                                        pts.append([j,y])
                                    leg_extra[i] = pts
                                js['task7']['extrapolated'] = leg_extra
                            elif 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                                x_cors,y_cors = list(),list()
                                for i in js['task6']['output']['data series']:
                                    name = i['name']
                                    for j in i['data']:
                                        x_cors.append(j['x'])
                                        y_cors.append(j['y'])
                                extra = dict()
                                extra_pts = list()
                                for i in tick_int:
                                    y = extrapolate(x_cors,y_cors,i)
                                    extra_pts.append([i,y])
                                extra[name] = extra_pts
                                js['task7']['extrapolated'] = extra
                            if 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])>1:
                                leg_pts = dict()
                                leg_extra = dict()
                                for i in js['task7']['name_of_data_series']:
                                    x_coor,y_coor = list(),list()
                                    for j in js['task6']['output']['data series']:
                                        if j['name'] == i:
                                            for k in j['data']:
                                                x_coor.append(k['x'])
                                                y_coor.append(k['y'])
                                    leg_pts[i] = [x_coor,y_coor]
                                for i in leg_pts:
                                    x_cors,y_cors = leg_pts[i][0],leg_pts[i][1]
                                    pts = list()
                                    for j in tick_int:
                                        y = extrapolate(x_cors,y_cors,j)
                                        pts.append([j,y])
                                    leg_extra[i] = pts
                                js['task7']['extrapolated'] = leg_extra
                        except:
                            pass
        
        obj = json.dumps(js,indent = 4)
        with open(copy_folder+file,'w') as f:
            f.write(obj)